In [1]:
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms as tfs
from datetime import datetime
import torch.nn.functional as F

In [2]:
data_tf = tfs.ToTensor()

In [3]:
train_set = ImageFolder('./Fruit-Images-Dataset/Training', transform = data_tf)
test_set = ImageFolder('./Fruit-Images-Dataset/Test', transform = data_tf)

In [12]:
train_data = DataLoader(train_set, batch_size = 64, shuffle = True)
test_data = DataLoader(test_set, batch_size = 128, shuffle = False)

In [5]:
class conv_net1(nn.Module):
    def __init__(self):
        super(conv_net1,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.max_pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.max_pool2 = nn.MaxPool2d(2 ,2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.max_pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.batch_norm4 = nn.BatchNorm2d(256)
        self.max_pool4 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(6400, 3200)
        self.fc2 = nn.Linear(3200, 1600)
        self.fc3 = nn.Linear(1600, 101)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = self.max_pool3(x)
        x = self.conv4(x)
        x = self.batch_norm4(x)
        x = self.max_pool4(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [6]:
net = conv_net1()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

In [7]:
def set_learning_rate(optimizer, lr):
    for param_groups in optimizer.param_groups:
        param_groups['lr'] = lr
        
def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().data.item()
    return num_correct / total

def train(net, train_data, test_data, epoch, optimizer, criterion):
    train_losses = []
    test_losses = []
    if torch.cuda.is_available():
        net = net.cuda()
    prev_time = datetime.now()
    for epoch in range(epoch):
        if epoch == 15:
            set_learning_rate(optimizer, 0.001)
        train_loss = 0
        train_acc = 0
        net = net.train()
        for im, labels in train_data:
            if torch.cuda.is_available():
                im = im.cuda()
                labels = labels.cuda()
            
            output = net(im)
            loss = criterion(output, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.data.item()
            train_acc += get_acc(output, labels)
        cur_time = datetime.now()
        h,remainder = divmod((cur_time - prev_time).seconds, 3600)
        m,s = divmod(remainder, 60)
        time_str = 'Time:%02d:%02d:%02d'%(h, m, s)
        
        test_loss = 0
        test_acc = 0
        net = net.eval()
        for im, labels in test_data:
            if torch.cuda.is_available():
                im = im.cuda()
                labels = labels.cuda()
            output = net(im)
            loss = criterion(output, labels)
            test_loss += loss.data.item()
            test_acc += get_acc(output, labels)
        epoch_str = (
                "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), test_loss / len(test_data),
                   test_acc / len(test_data)))
        prev_time = cur_time
        train_losses.append(train_loss / len(train_data))
        test_losses.append(test_loss / len(test_data))
        print(epoch_str + time_str)

In [8]:
train(net, train_data, test_data, 25, optimizer, criterion)

Epoch 0. Train Loss: 2.566145, Train Acc: 0.678380, Valid Loss: 0.807374, Valid Acc: 0.820117, Time:00:01:00
Epoch 1. Train Loss: 0.463633, Train Acc: 0.879126, Valid Loss: 0.785649, Valid Acc: 0.867754, Time:00:01:13
Epoch 2. Train Loss: 0.351352, Train Acc: 0.914989, Valid Loss: 0.482884, Valid Acc: 0.903589, Time:00:01:13
Epoch 3. Train Loss: 0.215261, Train Acc: 0.945516, Valid Loss: 0.417356, Valid Acc: 0.921422, Time:00:01:13
Epoch 4. Train Loss: 0.149103, Train Acc: 0.959777, Valid Loss: 0.950699, Valid Acc: 0.857563, Time:00:01:13
Epoch 5. Train Loss: 0.130315, Train Acc: 0.965866, Valid Loss: 0.281978, Valid Acc: 0.939651, Time:00:01:13
Epoch 6. Train Loss: 0.085828, Train Acc: 0.976945, Valid Loss: 0.389917, Valid Acc: 0.932518, Time:00:01:13
Epoch 7. Train Loss: 0.389600, Train Acc: 0.922172, Valid Loss: 0.792895, Valid Acc: 0.902174, Time:00:01:13
Epoch 8. Train Loss: 0.162028, Train Acc: 0.962342, Valid Loss: 0.756856, Valid Acc: 0.877264, Time:00:01:13
Epoch 9. Train Loss